# ORPO Training Script
Training script for ORPO fine-tuning from a base model.

In [1]:
import torch
from datasets import Dataset
from trl import ORPOTrainer, ORPOConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd

/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# Empty VRAM cache
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

### Load Base Model and Tokenizer

In [3]:
model_type="VAGOsolutions/SauerkrautLM-1.5b"

tokenizer = AutoTokenizer.from_pretrained(model_type)
model = AutoModelForCausalLM.from_pretrained(model_type, device_map="auto", trust_remote_code=True)
model.to("cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear

### Add custom Special Tokens (Functional Tokens)

In [4]:
new_tokens = ["<oc_1>", "<oc_2>", "<oc_3>", "<oc_end>"]
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
new_tokens

{'<oc_1>', '<oc_2>', '<oc_3>', '<oc_end>'}

### Adapt Model Weights

In [5]:
tokenizer.add_tokens(list(new_tokens))
model.resize_token_embeddings(len(tokenizer))

Embedding(151650, 1536)

#### Add BOS Token
This step was for some reasons only necessary for Qwen2-based models.

In [6]:
tokenizer.add_special_tokens({"bos_token": tokenizer.eos_token})
tokenizer.bos_token_id = tokenizer.eos_token_id

### Load the ORPO Dataset

In [7]:
german_dpo_dataset = pd.read_csv("german-dpo-dataset.csv", sep=";", names=["prompt", "chosen", "rejected"], skiprows=1)
german_dpo_dataset

,prompt,chosen,rejected
0,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Training im Fitnessstudio"", ""2026-11-2...","<oc_1>(""Training inFitness Studio"", ""2026-11-2..."
1,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorlesung Mittelhochdeutsch"", ""2023-02...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2026-..."
2,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2025-...","<oc_1>(""VorbeprechbungBachelorzeit"", ""2024-09-..."
3,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Weihnachtsessen bei meinen Eltern"", ""2...","<oc_1>(""Weihnachtssuch"", ""2025-10-13"", ""12:00""..."
4,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""wöchentliches Teammeeting"", ""2023-12-2...","<oc_1>(""wöschtragenen Termin mit ID b867c9bdae..."
...,...,...,...
2995,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-10-13"", False)<oc_end>\nFunktions...","<oc_3>(""2025-10-13"", False)<oc_end>\nFunktions..."
2996,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2023-12-04"", True)<oc_end>\nFunktionsb...","<oc_3>(""2026-12-05"", True)<oc_end>\nFunktionsb..."
2997,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-12-07"", False)<oc_end>\nFunktions...","<oc_3>(""2026-12-07"", False)<oc_end>\nFunktions..."
2998,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions..."


### Preprocess the Dataset for ORPO Fine-Tuning

In [8]:
# german_dpo_dataset.chosen = german_dpo_dataset.chosen.apply(lambda x: x.split("\n")[0])

def reformat_rejected(row):
    chosen_parts = row["chosen"].split("\n")
    rejected_parts = row["rejected"].split("\n")
    chosen_parts[0] = rejected_parts[0]
    return "\n".join(chosen_parts)

german_dpo_dataset.rejected = german_dpo_dataset.apply(reformat_rejected, axis=1)
german_dpo_dataset.rejected.iloc[0]

'<oc_1>("Training inFitness Studio", "2026-11-28", "20:00", 21)<oc_end>\nFunktionsbeschreibung: def create_calender_entry(title, date, time, duration):\n"""\nErstellt einen Kalendereintrag mit den angegebenen Details und ruft die Kalender-API auf.\n\nParameter:\ntitle (str): Der Titel des Kalendereintrags.\ndate (str): Das Datum des Kalendereintrags im Format \'YYYY-MM-DD\'.\ntime (str): Die Uhrzeit des Kalendereintrags im Format \'HH:MM\'.\nduration (int): Die Dauer des Kalendereintrags in Minuten.\n\nRückgabe:\nbool: True, wenn der Aufruf der Kalender-API erfolgreich war, andernfalls False.\n"""'

In [9]:
# german_dpo_dataset.chosen = german_dpo_dataset.chosen.apply(lambda x: x.split("\n")[0])
# german_dpo_dataset.rejected = german_dpo_dataset.rejected.apply(lambda x: x.split("\n")[0])
# german_dpo_dataset

### Create a Subset of the Dataset

In [10]:
num_samples = 1000

sample1 = german_dpo_dataset.iloc[:1000].sample(n=num_samples, random_state=1)
sample2 = german_dpo_dataset.iloc[1000:2000].sample(n=num_samples, random_state=1)
sample3 = german_dpo_dataset.iloc[2000:].sample(n=num_samples, random_state=1)

# Combine the samples
result = pd.concat([sample1, sample2, sample3])

In [11]:
german_dpo_dataset

,prompt,chosen,rejected
0,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Training im Fitnessstudio"", ""2026-11-2...","<oc_1>(""Training inFitness Studio"", ""2026-11-2..."
1,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorlesung Mittelhochdeutsch"", ""2023-02...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2026-..."
2,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2025-...","<oc_1>(""VorbeprechbungBachelorzeit"", ""2024-09-..."
3,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Weihnachtsessen bei meinen Eltern"", ""2...","<oc_1>(""Weihnachtssuch"", ""2025-10-13"", ""12:00""..."
4,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""wöchentliches Teammeeting"", ""2023-12-2...","<oc_1>(""wöschtragenen Termin mit ID b867c9bdae..."
...,...,...,...
2995,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-10-13"", False)<oc_end>\nFunktions...","<oc_3>(""2025-10-13"", False)<oc_end>\nFunktions..."
2996,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2023-12-04"", True)<oc_end>\nFunktionsb...","<oc_3>(""2026-12-05"", True)<oc_end>\nFunktionsb..."
2997,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-12-07"", False)<oc_end>\nFunktions...","<oc_3>(""2026-12-07"", False)<oc_end>\nFunktions..."
2998,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions..."


In [12]:
dataset = Dataset.from_dict(result)
dataset[0]

{'prompt': 'Unten steht ein Befehl des Benutzer, bitte wähle die korrekte Funktion aus und generiere Parameter, um die Funktion aufzurufen.\nBefehl: Erstelle einen neuen Kalendareintrag für Weihnachtsessen bei meinen Eltern am 07.09.2025 um 12 Uhr für 155 Minuten\nAntwort: ',
 'chosen': '<oc_1>("Weihnachtsessen bei meinen Eltern", "2025-09-07", "12:00", 155)<oc_end>\nFunktionsbeschreibung: def create_calender_entry(title, date, time, duration):\n"""\nErstellt einen Kalendereintrag mit den angegebenen Details und ruft die Kalender-API auf.\n\nParameter:\ntitle (str): Der Titel des Kalendereintrags.\ndate (str): Das Datum des Kalendereintrags im Format \'YYYY-MM-DD\'.\ntime (str): Die Uhrzeit des Kalendereintrags im Format \'HH:MM\'.\nduration (int): Die Dauer des Kalendereintrags in Minuten.\n\nRückgabe:\nbool: True, wenn der Aufruf der Kalender-API erfolgreich war, andernfalls False.\n"""',
 'rejected': '<oc_1>("Weihnachtssuch", "2024-09-07", "12:00", 163)<oc_end>\nFunktionsbeschreibun

### Train the Model

In [13]:
orpo_config = ORPOConfig(
    output_dir="german-function-calling-orpo",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    optim = "adamw_8bit",
    save_strategy="epoch",
    logging_steps = 10,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    warmup_steps=10,
    beta=0.1, # the lambda/alpha hyperparameter in the paper/code
    seed = 3407,
)

orpo_trainer = ORPOTrainer(
    model,
    args=orpo_config,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:209: UserWarning: `max_length` is not set in the ORPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:218: UserWarning: `max_prompt_length` is not set in the ORPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 3000/3000 [00:04<00:00, 617.84 examples/s]
[codecarbon INFO @ 10:17:09] [setup] RAM Tracking...
[codecarbon INFO @ 10:17:09] [setup] GPU Tracking...
[codecarbon 

In [14]:
orpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,1.166200
20,0.435900
30,0.311000
40,0.226700
50,0.240200
60,0.230700
70,0.207400
80,0.203800
90,0.158600
100,0.200600


[codecarbon INFO @ 10:17:27] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 10:17:27] Energy consumed for all GPUs : 0.001025 kWh. Total GPU Power : 245.86316082719773 W
[codecarbon INFO @ 10:17:27] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:17:27] 0.001251 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:17:42] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 10:17:42] Energy consumed for all GPUs : 0.002061 kWh. Total GPU Power : 248.90569358791834 W
[codecarbon INFO @ 10:17:42] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:17:42] 0.002513 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:17:57] Energy consumed for RAM : 0.000147 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 10:17:57] Energy consumed for all GPUs : 0.003103 kWh. Total GPU Power : 250.20001568

TrainOutput(global_step=4500, training_loss=0.11652086611588797, metrics={'train_runtime': 1865.1893, 'train_samples_per_second': 4.825, 'train_steps_per_second': 2.413, 'total_flos': 0.0, 'train_loss': 0.11652086611588797, 'epoch': 3.0})

### Save the Model and the Tokenizer

In [15]:
orpo_trainer.save_model("german-function-calling-orpo-model")

In [16]:
tokenizer.save_pretrained("german-function-calling-orpo-model")

('german-function-calling-orpo-model/tokenizer_config.json',
 'german-function-calling-orpo-model/special_tokens_map.json',
 'german-function-calling-orpo-model/vocab.json',
 'german-function-calling-orpo-model/merges.txt',
 'german-function-calling-orpo-model/added_tokens.json',
 'german-function-calling-orpo-model/tokenizer.json')

### Model Inference Examples

In [17]:
from transformers import GenerationConfig

# prompt = """Below is the query from the users, please choose the correct function and generate the
# parameters to call the function.
# Query: Create an appointment 'Business Lunch' on 2024-06-21 at 10:00 for 120 minutes.
# Response: """

prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: {query}
Antwort: """.format(query="Liste alle meine noch offenen Kalendareinträge für den 24.06.2024 auf")

early_stopping_token = "<oc_end>"
eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
eos_token_id

generation_config = GenerationConfig(
    penalty_alpha=0.0,
    do_sample=True,
    top_k=1,
    temperature=0.1,
    repetition_penalty=2.0,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=eos_token_id  # Set the early stopping token ID
)

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: Liste alle meine noch offenen Kalendareinträge für den 24.06.2024 auf
Antwort: <oc_3>("202 four-06-24", True)<oc_end>


In [18]:
from transformers import GenerationConfig

# prompt = """Below is the query from the users, please choose the correct function and generate the
# parameters to call the function.
# Query: Create an appointment 'Business Lunch' on 2024-06-21 at 10:00 for 120 minutes.
# Response: """

prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: {query}
Antwort: """.format(query="Erstelle einen neuen Kalendareintrag für eine Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten")

early_stopping_token = "<oc_end>"
eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
eos_token_id

generation_config = GenerationConfig(
    penalty_alpha=0.0,
    do_sample=True,
    top_k=1,
    temperature=0.1,
    repetition_penalty=2.0,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=eos_token_id  # Set the early stopping token ID
)

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: Erstelle einen neuen Kalendareintrag für eine Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten
Antwort: <oc_1>("Projektpräsentation", "YYYY-MM-DD")<oc_end>


In [19]:
from transformers import GenerationConfig

# prompt = """Below is the query from the users, please choose the correct function and generate the
# parameters to call the function.
# Query: Create an appointment 'Business Lunch' on 2024-06-21 at 10:00 for 120 minutes.
# Response: """

prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: {query}
Antwort: """.format(query="Lösche den Kalendareintrag mit ID 39asdaf972")

early_stopping_token = "<oc_end>"
eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
eos_token_id

generation_config = GenerationConfig(
    penalty_alpha=0.0,
    do_sample=True,
    top_k=1,
    temperature=0.1,
    repetition_penalty=2.0,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=eos_token_id  # Set the early stopping token ID
)

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: Lösche den Kalendareintrag mit ID 39asdaf972
Antwort: <oc_2>("39abdac9   1")<oc_end>
